In [8]:
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
from isaacgym import gymtorch
from isaacgym import gymapi


from isaacgymenvs import train
import isaacgymenvs
from rl_games.torch_runner import Runner

from omegaconf import OmegaConf
from hydra import compose, initialize
import yaml
import torch
import numpy as np
import time
import os

Importing module 'gym_37' (/scr-ssd/ksrini/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_37.so)
Setting GYM_USD_PLUG_INFO_PATH to /scr-ssd/ksrini/Downloads/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 1.8.1
Device count 1
/scr-ssd/ksrini/Downloads/isaacgym/python/isaacgym/_bindings/src/gymtorch
Using /afs/cs.stanford.edu/u/ksrini/.cache/torch_extensions as PyTorch extensions root...
Emitting ninja build file /afs/cs.stanford.edu/u/ksrini/.cache/torch_extensions/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module gymtorch...


In [2]:
os.environ['DISPLAY'] = ":4"

In [3]:
with initialize(config_path="cfg", job_name="test_env"):
    cfg = compose(config_name="config", overrides=["task=AllegroHandGrasp", 
                                                   "task.env.observationType=full",
                                                   # "task.env.objectType=spray_bottle",
                                                   "sim_device=cpu", 
                                                   "test=true",
                                                   # "task.env.useRelativeControl=true",
                                                   # "checkpoint=./runs/scaled-ac/nn/scaled-ac.pth",
                                                   "num_envs=2"])

env = isaacgymenvs.make(cfg.seed, cfg.task, cfg.num_envs, cfg.sim_device,
                  cfg.rl_device, cfg.graphics_device_id, cfg.headless,
                  cfg.multi_gpu, cfg.capture_video, cfg.force_render, cfg)

/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/hydra/_internal/defaults_list.py:413: UserWarning: In config: Invalid overriding of hydra/job_logging:
Default list overrides requires 'override' keyword.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/defaults_list_override for more information.

  deprecation_warning(msg)
2022-07-13 07:04:23,666 - INFO - logger - logger initialized
/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
  "Box bound precision lowered by casting to {}".format(self.dtype)


Error: FBX library failed to load - importing FBX data will not succeed. Message: No module named 'fbx'
FBX tools must be installed from https://help.autodesk.com/view/FBX/2020/ENU/?guid=FBX_Developer_Help_scripting_with_python_fbx_installing_python_fbx_html
Obs type: full
GPU Pipeline can only be used with GPU simulation. Forcing CPU Pipeline.
Not connected to PVD
Physics Engine: PhysX
Physics Device: cpu
GPU Pipeline: disabled
Num hand dofs:  17
Max effort:  1.0
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Max effort:  0.35
Num dofs:  19


## Get camera transform

Saving camera transform from current viewer

```python
import pickle
transform = env.gym.get_viewer_camera_transform(env.viewer, env.envs[0])
pos = transform.p
print("position", pos.x, pos.y, pos.z)

rot = transform.r
print("rotation", rot.x, rot.y, rot.z, rot.w)
pickle.dump(transform, open('allegro_viewer.pkl', 'wb'))
```

```python
import pickle

transform = pickle.load(open('allegro_viewer.pkl', 'rb'))
cam_pos = transform.p
cam_target = gymapi.Vec3()
env.gym.viewer_camera_look_at(env.viewer, None, cam_pos, cam_target)
```

In [4]:
from isaacgym import torch_utils

def get_action(t):
    actions =  torch_utils.unscale(env.shadow_hand_dof_default_pos.repeat(2, 1),
                                   env.shadow_hand_dof_lower_limits, env.shadow_hand_dof_upper_limits).numpy()
    t += 6
    scale = .5 if t > 250 else .25
    actions[:, 0] = - (np.sin(t-6/250 * np.pi))
    return torch.tensor(actions, device=env.device).float()

In [5]:
import pickle
transform = pickle.load(open('allegro_viewer.pkl', 'rb'))
cam_pos = gymapi.Vec3()
cam_pos.x = 0.4
cam_pos.y = -0.75
cam_pos.z = 1.5
cam_target = gymapi.Vec3()
cam_target.x = 0.4
cam_target.y = 0.4
env.gym.viewer_camera_look_at(env.viewer, None, cam_pos, cam_target)

In [5]:
env.reset_idx([0, 1], [0, 1])  # resets
print(env.shadow_hand_dof_pos[0])
action = get_action(-6)
obs, r, done, info = env.step(action)

print(env.object_pos)
print(env.object_rot)
print(env.shadow_hand_dof_pos[:, 0])

tensor([-0.0410, -0.2629,  0.0542,  0.0377, -0.0363, -0.1162,  0.0213,  0.0213,
         0.0378, -0.1116,  0.0378,  0.0378,  0.0213,  0.7429,  0.0062, -0.2792,
        -0.2792])
tensor([[-0.1224, -0.0812,  0.1236],
        [-0.1224, -0.0812,  0.1236]])
tensor([[ 0.0150, -0.0170, -0.0013,  0.9997],
        [ 0.0150, -0.0171, -0.0013,  0.9997]])
tensor([-0.0413, -0.0413])


In [7]:
t = 0
hand_vels, palm_pos = [], []
dof_pos = []
# moves hand up and down
while True:
# while t < 500:
    t += 1
    action = get_action(-6)
    obs, r, done, info = env.step(action)
    env_ids = done.nonzero(as_tuple=False).squeeze(-1)
    goal_env_ids = env.reset_goal_buf.nonzero(as_tuple=False).squeeze(-1)
    env.object_dof_vel[:] = 0
    env.object_linvel[:] = 0
    env.object_angvel[:] = 0
    # time.sleep(0.01)
    if done.any():
        env.reset_idx(env_ids, goal_env_ids)
    # palm_1_pos = env.rigid_body_states[0, 1][:3].detach().cpu().numpy().copy()
    # palm_2_pos = env.rigid_body_states[1, 1][:3].detach().cpu().numpy().copy()
    # dof_pos.append(env.shadow_hand_dof_pos[:, 0].detach().cpu().numpy().copy())
    # palm_pos.append((palm_1_pos, palm_2_pos))
    # hand_vels.append(env.shadow_hand_dof_vel[:, 0].detach().cpu().numpy().copy())

SystemExit: 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot([x[0][-1] for x in palm_pos])
plt.plot([x[1][-1] for x in palm_pos])
plt.plot([x[0] for x in dof_pos])
plt.plot([x[1] for x in dof_pos])

In [ ]:
plt.plot(np.array(hand_vels)[:100])